# Proxy pass

In my experience, proxying requests is the most common use of the `location` directive, making the `proxy_pass` directive crucial in this context. This page explores various options associated with the `proxy_pass` directive.

Check corresponding [official documentation section](https://nginx.org/en/docs/http/ngx_http_proxy_module.html#proxy_pass).

This section features a slightly complex setup to minimize code duplication during experiments. It uses its own Docker container, created in the following cell.

In [60]:
docker run --rm -it --name nginx_proxy_pass -p 80:80 -itd nginx

c94cfdce5103d6e493d8df5e9c767cdfc35a7581f4ba7ce0a0c377bca7ff161f


It will have the main server listening on port 80, and a mirror server on port 81 that simply returns the URL it was requested with. This setup makes it easy to monitor how different configurations transform requests generated by NGINX in various cases.

In [70]:
docker exec -i nginx_proxy_pass sh -c 'cat >  /etc/nginx/nginx.conf' <<EOF
events {}
http {

    server {
        listen 80;
        location / {
            include conf.d/default.conf;
        }
        
    }

    server {
        listen 81;
        location / {return 200  "\$scheme://\$host\$request_uri";}
    }
}
EOF

The previous configuration referred to the `conf.d/default.conf` file, where you can experiment with various `proxy_pass` configurations. The following cell sets up the proxy on port 80 to forward requests to the mirror server.

In [71]:
docker exec -i nginx_proxy_pass sh -c 'cat >  /etc/nginx/conf.d/default.conf' <<EOF
proxy_pass "http://localhost:81";
EOF
docker exec -i nginx_proxy_pass nginx -s reload &> /dev/null

Now, let's verify how it functions:

In [68]:
curl localhost:80/test

http://localhost/test


Referring to the `/test` uri on `localhost` returns `http://localhost/test` - expected behavior.

Don't forget to clean up the environment after everything is done.

In [76]:
docker stop nginx_proxy_pass

nginx_proxy_pass


## URI to upstream

In [36]:
docker exec -i experiment_nginx sh -c 'cat >  /etc/nginx/nginx.conf' <<EOF
events {}
http {
    server {
        listen 80;
        location / {return 200  "\$scheme://\$host\$request_uri";}
    }

    server {
        listen 81;
        location /proxy {
            proxy_pass "http://localhost:80";
        }
    }
}
EOF
docker exec -i experiment_nginx nginx -s reload &> /dev/null
docker exec -it experiment_nginx curl localhost:81/proxy/test

http://localhost/proxy/test


In [39]:
docker exec -i experiment_nginx sh -c 'cat >  /etc/nginx/nginx.conf' <<EOF
events {}
http {
    server {
        listen 80;
        location / {return 200  "\$scheme://\$host\$request_uri";}
    }

    server {
        listen 81;
        location /proxy/ {
            proxy_pass "http://localhost:80/";
        }
    }
}
EOF
docker exec -i experiment_nginx nginx -s reload &> /dev/null
docker exec -it experiment_nginx curl localhost:81/proxy/test

http://localhost/test


In [38]:
docker exec -i experiment_nginx sh -c 'cat >  /etc/nginx/nginx.conf' <<EOF
events {}
http {
    server {
        listen 80;
        location / {return 200  "\$scheme://\$host\$request_uri";}
    }

    server {
        listen 81;
        location /proxy {
            proxy_pass "http://localhost:80/wow";
        }
    }
}
EOF
docker exec -i experiment_nginx nginx -s reload &> /dev/null
docker exec -it experiment_nginx curl localhost:81/proxy/test

http://localhost/wow/test


## Wrong pass

Note that if you try to specify a non-existent path in the `proxy_pass` directive, it will lead to an error during configuration validation.

---

The following cell defines new nginx config that uses url `http://this_pass_doesnt_exist` as `proxy_pass` which is obviously wrong.

In [75]:
docker exec -i nginx_proxy_pass sh -c 'cat >  /etc/nginx/conf.d/default.conf' <<EOF
proxy_pass "http://this_pass_doesnt_exist";
EOF
docker exec -i nginx_proxy_pass nginx -t | true

2024/12/13 12:06:30 [emerg] 170#170: host not found in upstream "this_pass_doesnt_exist" in /etc/nginx/conf.d/default.conf:1
nginx: [emerg] host not found in upstream "this_pass_doesnt_exist" in /etc/nginx/conf.d/default.conf:1
nginx: configuration file /etc/nginx/nginx.conf test failed


During the validation of this config, we receive an error indicating that the specified host does not exist. This issue can cause problems when starting the container.